# Introduction
- Gradio is a UI-building library that enables quick prototyping and deployment of ML applications.
- It was originally a startup and later acquired by Hugging Face.
- The main goal is to create delightful and interactive ML apps with minimal effort.

# Examples

In [3]:
def shout(text):
    """
    This function takes a string and returns it in uppercase with an exclamation mark at the end.
    """
    return text.upper() + "!"

In [4]:
import gradio as gr

In [5]:
gr.Interface(
    fn=shout,
    inputs=gr.Textbox(label="Enter text to shout:"),
    outputs=gr.Textbox(label="Response:"),
    title="Shout It Out!",
    description="Type something and hear it shouted back at you!",
).launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv


## Flag
- The `flag` parameter is used in the flagging system, which allows users to save specific inputs and outputs from an interface. 
- This is useful for data collection, debugging, and tracking model errors.
- Flagged data is saved in a CSV file (by default) in the working directory.
- Default is enabled. You can disable it by passing `allow_flagging` as False.
- Modes
  - `manual` → Users manually click the flag button.
  - `auto` → Automatically flags every input-output pair.
  - `never` → Disables flagging.

In [6]:
from openai import OpenAI
from dotenv import load_dotenv

In [15]:
load_dotenv()
openai = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="ollama")

In [16]:
system_prompt = "You are a helpful assistant."

In [17]:
def message_gpt(prompt, model):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": prompt },
        ]
    )
    return response.choices[0].message.content

In [22]:
from matplotlib.pyplot import flag


gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Enter your message:"), gr.Dropdown(label="Select model:", choices=["llama3.2", "gpt-4o"])],
    outputs=gr.Textbox(label="Response from GPT:"),
    title="Chat with GPT",
    description="Type a message and get a response from the GPT model.",
    allow_flagging="never"
).launch()

c:\TFS\Study\machine-learning\.venv\lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


## Streaming with Gradio

In [30]:
def stream_with_gpt(prompt, model):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": prompt },
        ],
        stream=True
    )
    
    # Note - Below is the standard code for getting streamed chunks from the response. 
    #for chunk in response:
    #    yield chunk.choices[0].delta.content or ""

    # However, gradio expects full content to be returned, so we need to collect the chunks 
    # and return them as a generator (using yield).
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [31]:
gr.Interface(
    fn=stream_with_gpt,
    inputs=[gr.Textbox(label="Enter your message:"), gr.Dropdown(label="Select model:", choices=["llama3.2", "gpt-4o"])],
    outputs=gr.Textbox(label="Response from GPT:"),
    title="Stream with GPT",
    description="Type a message and get a streamed response from the GPT model.",
    allow_flagging="never",
).launch()

c:\TFS\Study\machine-learning\.venv\lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


## Summarize company brochure with Gradio and Markdown

In [34]:
import requests
from bs4 import BeautifulSoup
class Website:
    def __init__(self, url):
        self.url = url

    def get_website_contents(self):
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(['script', 'style', 'img', 'input']):
            irrelevant.decompose()
        content = soup.body.get_text(separator='\n', strip=True)
        return title, content

In [35]:
def get_brochure(url, model):
    website = Website(url)
    title, content = website.get_website_contents()

    system_prompt = f"""
    You are an expert in creating company brochure given the URLs and content of it. 
    You create this brochure and return it in Markdown format."""
    
    user_prompt = f"""
        The website you have been provided is of {website.url}.
        The title of the website is {title}.
        The content of the website is as follows:
        {content}
        Using this data, you need to create a creative company brochure in Markdown format.
    """

    
    response = openai.chat.completions.create(
        model=model,
        messages=[
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": user_prompt },
        ]
    )
    return response.choices[0].message.content    

In [37]:
brochure_content = get_brochure("https://www.ollama.com", "llama3.2")
print(brochure_content)

**Ollama: Unlocking the Power of Large Language Models**

[Cover Image: A futuristic illustration of Llama, surrounded by code and connected devices]

Welcome to Ollama

We empower developers, researchers, and innovators to unlock the full potential of large language models. Our cutting-edge technology provides a range of tools and frameworks to help you build, customize, and deploy your own models.

**Our Models**
---------------

At Ollama, we are committed to delivering high-quality models that meet your specific needs. Our current offerings include:

* **Llama 3.3**: A state-of-the-art model for natural language processing and generation.
* **DeepSeek-R1**: A specialized model for deep learning applications.
* **Phi-4**: A general-purpose model for a wide range of NLP tasks.
* **Mistral**: A high-performance model for rapid prototyping and development.
* **Gemma 3**: Our latest breakthrough in language modeling, designed for exceptional accuracy and speed.

**Key Features**
-------

In [39]:
gr.Interface(
    fn=get_brochure,
    inputs=[gr.Textbox(label="Enter the URL of the website:"), gr.Dropdown(label="Select model:", choices=["llama3.2", "gpt-4o"])],
    outputs=gr.Textbox(label="Brochure Content:"),
    title="Create Company Brochure",
    description="Enter a URL and get a company brochure in Markdown format.",
    allow_flagging="never",
).launch()

c:\TFS\Study\machine-learning\.venv\lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


In [43]:
def get_brochure_stream(url, model):
    website = Website(url)
    title, content = website.get_website_contents()

    system_prompt = f"""
    You are an expert in creating company brochure given the URLs and content of it. 
    You create this brochure and return it in Markdown format."""
    
    user_prompt = f"""
        The website you have been provided is of {website.url}.
        The title of the website is {title}.
        The content of the website is as follows:
        {content}
        Using this data, you need to create a creative company brochure in Markdown format.
    """

    
    response = openai.chat.completions.create(
        model=model,
        messages=[
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": user_prompt },
        ],
        stream=True
    )

    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        result = result.replace("```", "").replace("markdown", "")
        yield result

In [44]:
gr.Interface(
    fn=get_brochure_stream,
    inputs=[gr.Textbox(label="Enter the URL of the website:"), gr.Dropdown(label="Select model:", choices=["llama3.2", "gpt-4o"])],
    outputs=gr.Markdown(label="Brochure Content:"),
    title="Create Company Brochure",
    description="Enter a URL and get a company brochure in Markdown format.",
    allow_flagging="never",
).launch()

c:\TFS\Study\machine-learning\.venv\lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.
